In [1]:
import pandas
import datetime
import feather
import numpy
import scipy.optimize
from aacgmv2 import convert_mlt
import seaborn as sns
from davitpy import utils
from imagers.ssusi import ssusi_utils
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib.dates import date2num, DateFormatter
%matplotlib inline

In [2]:
# Some constants
eventDate = datetime.datetime(2016,7,25)
timeStart = datetime.datetime(2016,7,25,4,0)
timeEnd = datetime.datetime(2016,7,25,6,30)
sapsTime = datetime.datetime(2016,7,25,5,30)
sapsRadList = [ "cvw", "cve","fhw",\
               "fhe", "bks", "wal" ]
selBeam = 13
sapsMlatRange = [ 52., 63. ]#[ 56., 60. ]
# we'll use a higher cutoff for SAID
saidVelCutoff = 200.
sapsNpointsCutoff = 0 # per radar per beam
velScale = [ -800, 800 ]
coords="mag"
pwrScale = [ 0, 40 ]
spwScale = [ 0, 100 ]

In [3]:
def round2(number):
    """
    Round a number to the closest half integer.
    """
    return round(number * 2) / 2

def get_west_vel(row):
    westVel = row['vLos']/( numpy.cos(\
                 numpy.deg2rad( 90.-\
                    row['azimCalcMag'] ) ) )
    if westVel >= 2000.:
        return numpy.nan
    return westVel


def convert_to_datetime(row):
    currDateStr = str( int( row["dateStr"] ) )
#     return currDateStr
    if row["timeStr"] < 10:
        currTimeStr = "000" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 100:
        currTimeStr = "00" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 1000:
        currTimeStr = "0" + str( int( row["timeStr"] ) )
    else:
        currTimeStr = str( int( row["timeStr"] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )


def get_mlon_from_mlt(row):
    if row["normMlt"] < 0: 
        currMLT = row["normMlt"] + 24.
    else:
        currMLT = row["normMlt"]
    if row["endPtnormMlt"] < 0: 
        currEndptMLT = row["endPtnormMlt"] + 24.
    else:
        currEndptMLT = row["endPtnormMlt"]
    row["Mlon"] = numpy.round( \
                        convert_mlt( currMLT, row["date"] , m2a=True ) )
    if row["Mlon"] > 180.:
        row["Mlon"] -= 360.
    row["EndptMlon"] = numpy.round( \
                        convert_mlt( currEndptMLT, row["date"] , m2a=True ) )
    if row["EndptMlon"] > 180.:
        row["EndptMlon"] -= 360.
    return row

In [4]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-vps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF["velMagn"] = numpy.abs(velsDF["vLos"])
# select data within a MLAT range
velsDF = velsDF[ (velsDF["MLAT"] >= sapsMlatRange[0]) &\
               (velsDF["MLAT"] <= sapsMlatRange[1]) ]
# filter SAID data
# we'll get only those velocities
# which have a westward sense and
# which are greater than cutoff (300 m/s)
velsDF = velsDF[ (numpy.abs(velsDF["vLos"]) >= saidVelCutoff) &\
               ( (velsDF["vLos"]/velsDF["azimCalcMag"]) > 0.)\
               ].reset_index(drop=True)
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,power,spwdth,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,velMagn
0,20160725,222,12,51,73.4178,306.4436,13.2653,60.1981,62.8753,-20.9717,20.1108,51.9185,-90.6409,207,cve,2016-07-25 02:22:00,-3.8892,306.4436
1,20160725,222,12,52,74.3175,338.2497,13.2653,65.5262,62.9828,-20.0068,20.1751,52.0017,-89.9959,207,cve,2016-07-25 02:22:00,-3.8249,338.2497
2,20160725,224,12,51,73.4178,327.2770,13.2653,65.5262,62.8753,-20.9717,20.1426,51.9185,-90.6409,207,cve,2016-07-25 02:24:00,-3.8574,327.2770
3,20160725,224,12,52,74.3175,338.2497,13.4738,65.5262,62.9828,-20.0068,20.2069,52.0017,-89.9959,207,cve,2016-07-25 02:24:00,-3.7931,338.2497
4,20160725,226,12,50,72.5254,318.6452,13.6400,79.2065,62.7607,-21.9288,20.1105,51.8318,-91.2833,207,cve,2016-07-25 02:26:00,-3.8895,318.6452


In [5]:
# # sel data from a radar
# rtiDF = velsDF[ (velsDF["radCode"] == sapsRadList[0]) &\
#               (velsDF["beam"] == selBeam) ]
# plotParam = "power"
# if plotParam == "vLos":
#     selParamScale = velScale
#     label = "L-o-S velocity [m/s]"
# elif plotParam == "power":
#     selParamScale = pwrScale
#     label = "Power"
# else:
#     selParamScale = spwScale
#     label = "Spectral Width [m/s]"
# # Make an RTIish Plot
# # Seaborn styling
# sns.set_style("whitegrid")
# sns.set_context("paper")
# seaMap = ListedColormap(sns.color_palette("Reds"))
# # Plot the data
# fig = plt.figure()
# ax = fig.add_subplot(111)
# rtiDF["dtNum"] = [ date2num(x) for x in rtiDF["date"] ]
# plotDF = rtiDF[ ["MLAT", "dtNum",\
#                         plotParam] ].pivot( "MLAT", "dtNum" )
# mlatVals = plotDF.index.values
# timeVals = plotDF.columns.levels[1].values

# timeCntr, mlatCntr  = numpy.meshgrid( timeVals, mlatVals )
# # Mask the nan values! pcolormesh can't handle them well!
# vLosVals = numpy.ma.masked_where(numpy.isnan(plotDF[plotParam].values),plotDF[plotParam].values)
# rtiPlot = ax.pcolormesh(timeCntr, mlatCntr, vLosVals, cmap=seaMap, vmin=selParamScale[0],vmax=selParamScale[1])
# # rtiPlot = ax.scatter( rtiDF["date"].values, rtiDF["MLAT"].values,\
# #            c=rtiDF[plotParam].values, cmap=seaMap,vmin=selParamScale[0], vmax=selParamScale[1], s=90. )

# ax.set_xlim( [ timeStart, timeEnd ] )
# ax.set_xlabel("TIME (UT)")
# ax.set_ylabel("MLAT")
# ax.get_xaxis().set_major_formatter(DateFormatter('%H:%M'))


# cbar = plt.colorbar(rtiPlot, orientation='vertical')
# cbar.set_label(label)

In [6]:
selCols = [ "date", "beam", "radCode",\
           "MLAT", "MLON", "power",\
           "spwdth", "vLos" ]
maxValsDF = velsDF[ selCols ].round()
maxValsDF = maxValsDF[ (maxValsDF["date"] >= timeStart) &\
                     (maxValsDF["date"] <= timeEnd) ]
# get max power
maxPwrGrp = maxValsDF[ [ "power", "MLON", "date" ]\
                  ].groupby( [ "date", "MLON" ]\
                ).max().reset_index()
maxPwrDF = pandas.merge( maxValsDF, maxPwrGrp,\
                        on=[ "date", "MLON", "power" ] )
# get max sp.wdth
maxSpwGrp = maxValsDF[ [ "spwdth", "MLON", "date" ]\
                  ].groupby( [ "date", "MLON" ]\
                ).max().reset_index()
maxSpwDF = pandas.merge( maxValsDF, maxSpwGrp,\
                        on=[ "date", "MLON", "spwdth" ] )

In [7]:
maxSpwDF.head()

,date,beam,radCode,MLAT,MLON,power,spwdth,vLos
0,2016-07-25 04:04:00,7,cve,61.0,-45.0,11.0,66.0,299.0
1,2016-07-25 04:04:00,7,cve,61.0,-44.0,12.0,66.0,299.0
2,2016-07-25 04:06:00,9,cve,61.0,-42.0,16.0,76.0,419.0
3,2016-07-25 04:06:00,10,cve,61.0,-40.0,17.0,92.0,513.0
4,2016-07-25 04:08:00,8,cve,62.0,-42.0,15.0,88.0,621.0
